In [376]:
import pandas as pd
import numpy as np
import datetime

df = pd.DataFrame({
        'Hours': pd.date_range('2018-01-01', '2018-01-10', freq='1H', closed='left'),
        'Steps': np.random.randint(100,10000, size=9*24),
        }
     )

ids = []
for i in range(1, 217):
    ids.append(i%10 + 1)
    
df["Id"] = ids


# Add day for id 1
df = df.append({'Hours': datetime.datetime(2020, 2, 2), 'Steps': 2000, 'Id': 1}, ignore_index=True)

# Remove 5 days from id 10
id_10_indices = df.loc[df.Id == 10].index.values[:-5]
df = df[~df.index.isin(id_10_indices)]


In [402]:
class FilterOperator:
    
    def __init__(
        self,
        participant_id_column="id",
        ts_column="time",
        epoch_filter="",
        day_filter=None,
        filter_type="include"):
        
        self.participant_id_column = participant_id_column
        self.ts_column = ts_column
        self.epoch_filter = epoch_filter
        self.day_filter = day_filter
        self.filter_type= filter_type    
   
    def process(self, *data_frames):
        
        processed = []
        
        for data_frame in data_frames:
            processed_df = None
   
            if type(self.epoch_filter) == str:
                if self.filter_type == "include":
                    processed_df = data_frame.query(self.epoch_filter)
                else:
                    processed_df = data_frame.query(f"not ({self.epoch_filter})")
            else:
                if self.filter_type == "include":       
                    processed_df = data_frame[self.epoch_filter(data_frame)]                     
                else:
                    processed_df = data_frame[~self.epoch_filter(data_frame)]

            if self.day_filter:

                if type(self.day_filter) == dict:            
                    index = processed_df.groupby([
                        self.participant_id_column, 
                        processed_df[self.ts_column].dt.date])[self.day_filter['column']].transform(
                            self.day_filter['filter'])
                    
                    if filter_type == "include":
                        processed_df = processed_df.loc[index]
                    else:
                        processed_df = processed_df.loc[~index]
                        
                    if ('consecutive_days' in self.day_filter) and not processed_df.empty:
                        processed_df = self.consecutive_days_filter(processed_df, self.day_filter['consecutive_days'])


                elif type(self.day_filter == list):
                    for day_filter_item in self.day_filter:
                        index = processed_df.groupby([
                            self.participant_id_column, 
                            processed_df[self.ts_column].dt.date])[day_filter_item['column']].transform(
                                day_filter_item['filter'])
                
                        if self.filter_type == "include":
                            processed_df = processed_df.loc[index]
                        else:
                            processed_df = processed_df.loc[~index]
                
                        if ('consecutive_days' in day_filter_item) and not processed_df.empty:
                            processed_df = self.consecutive_days_filter(processed_df, day_filter_item['consecutive_days'])

            processed.append(processed_df)
            
        return processed
    
    def consecutive_days_filter(df, min_consecutive_days):
        days_per_id = df.groupby([self.participant_id_column, pd.Grouper(key=self.ts_column, freq='D')]).sum().reset_index()
        
        # Take the date difference between a row and its previous row
        # If the difference is not 1 day, we start a new label for the sequence using pd.cumsum()
        label_of_consecutive_days_per_id = days_per_id.groupby(self.participant_id_column)[self.ts_column].diff().dt.days.ne(1).cumsum()
        consecutive_days_per_id = days_per_id.groupby([self.participant_id_column, self.label_of_consecutive_days_per_id]).size().reset_index(level=1, drop=True)

        # Match the index with the sequence labels
        consecutive_days_per_id = consecutive_days_per_id.reset_index()
        consecutive_days_per_id.index = consecutive_days_per_id.index + 1

        # Find the sequence label that matches the criteria
        sequences_to_keep = consecutive_days_per_id[consecutive_days_per_id[0] > min_consecutive_days].index.values

        # Find days to keep: Given the day sequence labels, include the one's in sequences_to_keep
        days_to_keep = days_per_id[label_of_consecutive_days_per_id.isin(sequences_to_keep)]

        df = df.groupby(self.participant_id_column).apply(lambda x: self.keep_days_in_df(x, days_to_keep)).reset_index(drop=True)
        return df


    def keep_days_in_df(df, days_to_keep):
        # Helper function for consecutive_days_filter
        days = days_to_keep.loc[days_to_keep[self.participant_id_column] == df.name, self.ts_column].dt.date
        return df[df[self.ts_column].dt.date.isin(days)]

        

def filter_operator(
    data_frame,    
    participant_id_column="id",
    ts_column="time",
    epoch_filter="",
    day_filter=None,
    filter_type="include"):
    
    processed_df = None
   
    if type(epoch_filter) == str:
        if filter_type == "include":
            processed_df = data_frame.query(epoch_filter)
        else:
            processed_df = data_frame.query(f"not ({epoch_filter})")
    else:
        if filter_type == "include":       
            processed_df = data_frame[epoch_filter(df)] 
        else:
            processed_df = data_frame[~epoch_filter(df)]

    if day_filter:
        
        if type(day_filter) == dict:            
            index = processed_df.groupby([
                participant_id_column, 
                processed_df[ts_column].dt.date])[day_filter['column']].transform(
                    day_filter['filter'])
            if filter_type == "include":
                processed_df = processed_df.loc[index]
            else:
                processed_df = processed_df.loc[~index]
            
            print(processed_df)
            if ('consecutive_days' in day_filter) and not processed_df.empty:
                processed_df = consecutive_days_filter(processed_df, day_filter['consecutive_days'])
                
        elif type(day_filter == list):
            for day_filter_item in day_filter:
                index = processed_df.groupby([
                    participant_id_column, 
                    processed_df[ts_column].dt.date])[day_filter_item['column']].transform(
                        day_filter_item['filter'])
                if filter_type == "include":
                    processed_df = processed_df.loc[index]
                else:
                    processed_df = processed_df.loc[~index]
                
                if ('consecutive_days' in day_filter_item) and not processed_df.empty:
                    processed_df = consecutive_days_filter(processed_df, day_filter_item['consecutive_days'])
            
    return processed_df


def consecutive_days_filter(df, min_consecutive_days):
    days_per_id = df.groupby(['Id', pd.Grouper(key='Hours', freq='D')]).sum().reset_index()
    
    
    # Take the date difference between a row and its previous row
    # If the difference is not 1 day, we start a new label for the sequence using pd.cumsum()
    label_of_consecutive_days_per_id = days_per_id.groupby('Id')['Hours'].diff().dt.days.ne(1).cumsum()
    consecutive_days_per_id = days_per_id.groupby(['Id', label_of_consecutive_days_per_id]).size().reset_index(level=1, drop=True)
    
    # Match the index with the sequence labels
    consecutive_days_per_id = consecutive_days_per_id.reset_index()
    consecutive_days_per_id.index = consecutive_days_per_id.index + 1
    
    # Find the sequence label that matches the criteria
    sequences_to_keep = consecutive_days_per_id[consecutive_days_per_id[0] > min_consecutive_days].index.values

    # Find days to keep: Given the day sequence labels, include the one's in sequences_to_keep
    days_to_keep = days_per_id[label_of_consecutive_days_per_id.isin(sequences_to_keep)]
    


    df = df.groupby('Id').apply(lambda x: keep_days_in_df(x, days_to_keep)).reset_index(drop=True)
    return df
    

def keep_days_in_df(df, days_to_keep):
    # Helper function for consecutive_days_filter
    days = days_to_keep.loc[days_to_keep['Id'] == df.name, 'Hours'].dt.date
    return df[df['Hours'].dt.date.isin(days)]

       

In [403]:
operator = FilterOperator(epoch_filter=lambda df: df['Steps'] < 2000, filter_type="exclude")
operator.process(df)


[                  Hours  Steps  Id
 0   2018-01-01 00:00:00   9867   2
 1   2018-01-01 01:00:00   2856   3
 2   2018-01-01 02:00:00   7532   4
 4   2018-01-01 04:00:00   3190   6
 5   2018-01-01 05:00:00   5468   7
 ..                  ...    ...  ..
 212 2018-01-09 20:00:00   3593   4
 213 2018-01-09 21:00:00   7631   5
 214 2018-01-09 22:00:00   6720   6
 215 2018-01-09 23:00:00   6301   7
 216 2020-02-02 00:00:00   2000   1
 
 [151 rows x 3 columns]]

In [416]:
df3 = filter_operator(
    df, 
    participant_id_column="Id",
    ts_column="Hours",
    epoch_filter=lambda df: df['Steps'] > 10, 
    day_filter={
        "column": "Hours",
        "filter": lambda x: x.count() < 10,
        "consecutive_days": 5
    },
    filter_type="include")

                  Hours  Steps  Id
0   2018-01-01 00:00:00   9867   2
1   2018-01-01 01:00:00   2856   3
2   2018-01-01 02:00:00   7532   4
3   2018-01-01 03:00:00    626   5
4   2018-01-01 04:00:00   3190   6
..                  ...    ...  ..
212 2018-01-09 20:00:00   3593   4
213 2018-01-09 21:00:00   7631   5
214 2018-01-09 22:00:00   6720   6
215 2018-01-09 23:00:00   6301   7
216 2020-02-02 00:00:00   2000   1

[201 rows x 3 columns]


In [417]:
df[df.Id == 10]

,Hours,Steps,Id
168,2018-01-08 00:00:00,8678,10
178,2018-01-08 10:00:00,3764,10
188,2018-01-08 20:00:00,7380,10
198,2018-01-09 06:00:00,943,10
208,2018-01-09 16:00:00,1747,10


In [418]:
df3

,Hours,Steps,Id
0,2018-01-01 09:00:00,4149,1
1,2018-01-01 19:00:00,8483,1
2,2018-01-02 05:00:00,7468,1
3,2018-01-02 15:00:00,126,1
4,2018-01-03 01:00:00,2089,1
...,...,...,...
190,2018-01-07 23:00:00,8322,9
191,2018-01-08 09:00:00,4355,9
192,2018-01-08 19:00:00,7090,9
193,2018-01-09 05:00:00,3130,9
